In [1]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [28]:
train_joke_df = pd.read_csv(r'data\recsys-in-practice\train_joke_df.csv')
test_joke_df_nofactrating = pd.read_csv(r'data\recsys-in-practice\test_joke_df_nofactrating.csv', index_col=0)

joke_df = pd.read_csv('joke_df_features_svd.csv')
user_df = pd.read_csv('user_df_features_svd.csv')

In [5]:
train_joke_df = train_joke_df.merge(user_df, left_on='UID', right_index=True).merge(joke_df, left_on='JID', right_index=True)
train_joke_df

,UID,JID,Rating,user_feature_1,user_feature_2,user_feature_3,user_feature_4,user_feature_5,user_feature_6,user_feature_7,...,joke_feature_91,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100
0,18029,6,-1.26,-0.058144,0.158959,0.015875,0.069882,0.161394,0.098246,0.120086,...,0.614931,-0.090240,0.003513,0.111881,-0.012335,-0.138197,0.458080,-0.541952,0.105341,0.147151
120304,3366,6,3.35,-0.276731,0.301820,-0.106114,0.084402,-0.022660,0.227516,0.061178,...,0.614931,-0.090240,0.003513,0.111881,-0.012335,-0.138197,0.458080,-0.541952,0.105341,0.147151
492994,12735,6,1.26,0.048716,0.123857,-0.122212,0.035329,0.024639,0.104828,0.096554,...,0.614931,-0.090240,0.003513,0.111881,-0.012335,-0.138197,0.458080,-0.541952,0.105341,0.147151
81927,11365,6,-5.87,0.039644,0.416506,-0.079063,0.004758,0.135379,0.372923,0.382700,...,0.614931,-0.090240,0.003513,0.111881,-0.012335,-0.138197,0.458080,-0.541952,0.105341,0.147151
650652,17990,6,-0.29,-0.070404,-0.053476,0.132871,0.133790,-0.257478,-0.195392,-0.296435,...,0.614931,-0.090240,0.003513,0.111881,-0.012335,-0.138197,0.458080,-0.541952,0.105341,0.147151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501169,19068,85,1.60,-0.067908,0.164236,0.052626,0.111260,0.248084,0.076495,0.209207,...,0.234221,-0.419674,-0.068509,0.260702,-0.094782,0.160607,0.178213,-0.137030,-0.014625,-0.246719
326575,24910,85,-1.80,-0.218189,0.306614,0.096680,0.226832,0.078665,0.010388,0.272278,...,0.234221,-0.419674,-0.068509,0.260702,-0.094782,0.160607,0.178213,-0.137030,-0.014625,-0.246719
1040976,9017,85,3.93,-0.017418,0.453289,0.001213,0.103245,0.329094,0.169336,0.578302,...,0.234221,-0.419674,-0.068509,0.260702,-0.094782,0.160607,0.178213,-0.137030,-0.014625,-0.246719
1148245,21968,85,3.54,-0.506287,0.317768,-0.057399,-0.011454,-0.274077,0.086048,-0.275787,...,0.234221,-0.419674,-0.068509,0.260702,-0.094782,0.160607,0.178213,-0.137030,-0.014625,-0.246719


In [29]:
test_joke_df_nofactrating = test_joke_df_nofactrating.merge(user_df, how='left', left_on='UID', right_index=True).merge(joke_df, how='left', left_on='JID', right_index=True)
test_joke_df_nofactrating

,UID,JID,user_feature_1,user_feature_2,user_feature_3,user_feature_4,user_feature_5,user_feature_6,user_feature_7,user_feature_8,...,joke_feature_91,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100
InteractionID,,,,,,,,,,,,,,,,,,,,,
0,11228,39,0.041494,-0.019669,-0.046502,0.003684,-0.088271,-0.067943,-0.074454,0.021834,...,-0.152084,-0.255151,-0.170399,0.300889,-0.094352,0.370475,-0.054978,0.089917,-0.048792,-0.003394
1,21724,85,-0.105730,0.600935,0.081735,0.280295,0.461382,0.055102,0.320269,-0.082147,...,0.234221,-0.419674,-0.068509,0.260702,-0.094782,0.160607,0.178213,-0.137030,-0.014625,-0.246719
2,16782,56,-0.046537,-0.124728,0.065352,0.036919,-0.228116,-0.152849,-0.376306,0.332513,...,-0.016803,-0.196697,0.186677,0.149783,-0.103317,0.576104,-0.114517,-0.063121,-0.169587,0.068116
3,12105,42,-0.125930,0.032269,0.031715,-0.065563,-0.155323,0.128054,-0.007486,-0.070116,...,-0.045566,-0.263738,-0.083895,0.261084,-0.148073,0.344654,-0.087479,-0.035381,-0.117311,-0.078073
4,14427,2,0.007909,-0.235814,0.025480,0.026753,-0.117487,-0.210771,-0.316912,0.068459,...,-0.162176,-0.297996,-0.132594,0.337949,0.067280,0.285934,-0.153183,0.132184,0.000751,-0.057890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362086,3085,66,0.117056,0.521257,-0.056028,0.179373,0.480985,0.191488,0.613380,0.081868,...,-0.213167,-0.388854,-0.223858,0.288515,-0.120058,0.255402,-0.131640,0.165418,0.017114,-0.151924
362087,13765,31,0.009805,-0.261898,0.203455,0.200472,0.025892,-0.362058,-0.187364,0.005845,...,0.549572,-0.104571,0.031827,0.070764,-0.249606,-0.049928,0.356879,-0.233594,-0.127793,-0.015678
362088,10341,29,-0.127947,-0.145092,-0.140193,-0.033378,-0.383222,-0.157345,-0.082459,-0.038048,...,0.289070,-0.246149,-0.237770,0.365300,-0.070434,0.254199,0.160938,-0.345331,0.097235,0.231820


In [9]:
test_joke_df_nofactrating.columns

Index(['UID', 'JID', 'user_feature_1', 'user_feature_2', 'user_feature_3',
       'user_feature_4', 'user_feature_5', 'user_feature_6', 'user_feature_7',
       'user_feature_8',
       ...
       'joke_feature_91', 'joke_feature_92', 'joke_feature_93',
       'joke_feature_94', 'joke_feature_95', 'joke_feature_96',
       'joke_feature_97', 'joke_feature_98', 'joke_feature_99',
       'joke_feature_100'],
      dtype='object', length=202)

In [10]:
train_joke_df["UID"] = train_joke_df["UID"].astype(str)
train_joke_df["JID"] = train_joke_df["JID"].astype(str)
test_joke_df_nofactrating["UID"] = test_joke_df_nofactrating["UID"].astype(str)
test_joke_df_nofactrating["JID"] = test_joke_df_nofactrating["JID"].astype(str)

In [11]:
train_df, valid_df = train_test_split(train_joke_df, test_size=0.1, random_state=42)

In [12]:
from catboost import CatBoostRanker, Pool, MetricVisualizer, CatBoostRegressor
from copy import deepcopy

In [13]:
cat_features = ['UID', 'JID']

In [14]:
train_pool = Pool(train_df.drop(columns='Rating'), label=train_df['Rating'], cat_features=cat_features)
valid_pool = Pool(valid_df.drop(columns='Rating'), label=valid_df['Rating'], cat_features=cat_features)
#main_pool = Pool(train_joke_df.drop(columns='Rating'), label=train_joke_df['Rating'], cat_features=cat_features)

#test_pool = Pool(test_joke_df_nofactrating[features], cat_features=cat_features)


In [12]:
#train = Pool(
#    data=X_train,
#    label=y_train,
#    group_id=queries_train,
#    cat_features=[0, 1]
#)

#test = Pool(
#    data=X_test,
#    label=y_test,
#    group_id=queries_test,
#    cat_features=[0, 1]
#)

In [15]:
default_parameters = {
    'iterations': 3000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE',
}


In [17]:
model = CatBoostRegressor(**default_parameters)
model.fit(train_pool, eval_set=valid_pool, plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.080269
0:	learn: 5.1438891	test: 5.1285152	best: 5.1285152 (0)	total: 453ms	remaining: 22m 38s
1:	learn: 5.0644487	test: 5.0427594	best: 5.0427594 (1)	total: 868ms	remaining: 21m 40s
2:	learn: 4.9962408	test: 4.9676688	best: 4.9676688 (2)	total: 1.18s	remaining: 19m 39s
3:	learn: 4.9366088	test: 4.9026197	best: 4.9026197 (3)	total: 1.48s	remaining: 18m 28s
4:	learn: 4.8850752	test: 4.8456284	best: 4.8456284 (4)	total: 1.77s	remaining: 17m 39s
5:	learn: 4.8398502	test: 4.7964014	best: 4.7964014 (5)	total: 2.07s	remaining: 17m 11s
6:	learn: 4.7994963	test: 4.7514751	best: 4.7514751 (6)	total: 2.35s	remaining: 16m 47s
7:	learn: 4.7656529	test: 4.7133792	best: 4.7133792 (7)	total: 2.65s	remaining: 16m 30s
8:	learn: 4.7356476	test: 4.6795208	best: 4.6795208 (8)	total: 2.94s	remaining: 16m 18s
9:	learn: 4.7097238	test: 4.6509249	best: 4.6509249 (9)	total: 3.3s	remaining: 16m 27s
10:	learn: 4.6869236	test: 4.6253472	best: 4.6253472 (10)	total: 3.55s	remaining: 16m 3s
11

92:	learn: 4.5140476	test: 4.4140426	best: 4.4140426 (92)	total: 34.3s	remaining: 17m 52s
93:	learn: 4.5139694	test: 4.4139601	best: 4.4139601 (93)	total: 34.7s	remaining: 17m 52s
94:	learn: 4.5137957	test: 4.4137609	best: 4.4137609 (94)	total: 35.3s	remaining: 17m 58s
95:	learn: 4.5137001	test: 4.4136845	best: 4.4136845 (95)	total: 35.6s	remaining: 17m 57s
96:	learn: 4.5136242	test: 4.4136209	best: 4.4136209 (96)	total: 36s	remaining: 17m 57s
97:	learn: 4.5135437	test: 4.4135598	best: 4.4135598 (97)	total: 36.3s	remaining: 17m 55s
98:	learn: 4.5134333	test: 4.4134804	best: 4.4134804 (98)	total: 36.7s	remaining: 17m 56s
99:	learn: 4.5132607	test: 4.4132539	best: 4.4132539 (99)	total: 37.2s	remaining: 17m 59s
100:	learn: 4.5131707	test: 4.4132166	best: 4.4132166 (100)	total: 37.6s	remaining: 17m 58s
101:	learn: 4.5131150	test: 4.4131905	best: 4.4131905 (101)	total: 38s	remaining: 17m 59s
102:	learn: 4.5129578	test: 4.4130134	best: 4.4130134 (102)	total: 38.6s	remaining: 18m 5s
103:	lear

182:	learn: 4.5046075	test: 4.4058082	best: 4.4058082 (182)	total: 1m 12s	remaining: 18m 32s
183:	learn: 4.5045309	test: 4.4057639	best: 4.4057639 (183)	total: 1m 12s	remaining: 18m 30s
184:	learn: 4.5044430	test: 4.4057010	best: 4.4057010 (184)	total: 1m 12s	remaining: 18m 30s
185:	learn: 4.5043613	test: 4.4056522	best: 4.4056522 (185)	total: 1m 13s	remaining: 18m 28s
186:	learn: 4.5042550	test: 4.4055836	best: 4.4055836 (186)	total: 1m 13s	remaining: 18m 26s
187:	learn: 4.5041624	test: 4.4055440	best: 4.4055440 (187)	total: 1m 13s	remaining: 18m 25s
188:	learn: 4.5040299	test: 4.4054159	best: 4.4054159 (188)	total: 1m 14s	remaining: 18m 24s
189:	learn: 4.5039258	test: 4.4053528	best: 4.4053528 (189)	total: 1m 14s	remaining: 18m 23s
190:	learn: 4.5037870	test: 4.4051277	best: 4.4051277 (190)	total: 1m 15s	remaining: 18m 25s
191:	learn: 4.5037052	test: 4.4050730	best: 4.4050730 (191)	total: 1m 15s	remaining: 18m 23s
192:	learn: 4.5036112	test: 4.4050262	best: 4.4050262 (192)	total: 1m 

271:	learn: 4.4946873	test: 4.3976341	best: 4.3976341 (271)	total: 1m 48s	remaining: 18m 4s
272:	learn: 4.4945435	test: 4.3974552	best: 4.3974552 (272)	total: 1m 48s	remaining: 18m 5s
273:	learn: 4.4944654	test: 4.3974285	best: 4.3974285 (273)	total: 1m 48s	remaining: 18m 4s
274:	learn: 4.4943869	test: 4.3973652	best: 4.3973652 (274)	total: 1m 49s	remaining: 18m 3s
275:	learn: 4.4943029	test: 4.3973183	best: 4.3973183 (275)	total: 1m 49s	remaining: 18m 2s
276:	learn: 4.4942405	test: 4.3972668	best: 4.3972668 (276)	total: 1m 50s	remaining: 18m 1s
277:	learn: 4.4941204	test: 4.3971883	best: 4.3971883 (277)	total: 1m 50s	remaining: 18m 1s
278:	learn: 4.4940497	test: 4.3971421	best: 4.3971421 (278)	total: 1m 50s	remaining: 18m 1s
279:	learn: 4.4939555	test: 4.3970970	best: 4.3970970 (279)	total: 1m 51s	remaining: 18m 3s
280:	learn: 4.4938416	test: 4.3970410	best: 4.3970410 (280)	total: 1m 51s	remaining: 18m 2s
281:	learn: 4.4936916	test: 4.3968829	best: 4.3968829 (281)	total: 1m 52s	remain

...

2637:	learn: 4.3655226	test: 4.3202979	best: 4.3202979 (2637)	total: 18m 5s	remaining: 2m 28s
2638:	learn: 4.3655030	test: 4.3202962	best: 4.3202962 (2638)	total: 18m 5s	remaining: 2m 28s
2639:	learn: 4.3654573	test: 4.3202911	best: 4.3202911 (2639)	total: 18m 5s	remaining: 2m 28s
2640:	learn: 4.3654081	test: 4.3202555	best: 4.3202555 (2640)	total: 18m 6s	remaining: 2m 27s
2641:	learn: 4.3653560	test: 4.3202210	best: 4.3202210 (2641)	total: 18m 6s	remaining: 2m 27s
2642:	learn: 4.3653184	test: 4.3201979	best: 4.3201979 (2642)	total: 18m 7s	remaining: 2m 26s
2643:	learn: 4.3652714	test: 4.3201857	best: 4.3201857 (2643)	total: 18m 7s	remaining: 2m 26s
2644:	learn: 4.3652381	test: 4.3201899	best: 4.3201857 (2643)	total: 18m 7s	remaining: 2m 26s
2645:	learn: 4.3652076	test: 4.3201791	best: 4.3201791 (2645)	total: 18m 8s	remaining: 2m 25s
2646:	learn: 4.3651921	test: 4.3201766	best: 4.3201766 (2646)	total: 18m 8s	remaining: 2m 25s
2647:	learn: 4.3651735	test: 4.3201713	best: 4.3201713 (2647

2724:	learn: 4.3622676	test: 4.3189328	best: 4.3189328 (2724)	total: 18m 42s	remaining: 1m 53s
2725:	learn: 4.3622331	test: 4.3189331	best: 4.3189328 (2724)	total: 18m 42s	remaining: 1m 52s
2726:	learn: 4.3621875	test: 4.3189158	best: 4.3189158 (2726)	total: 18m 42s	remaining: 1m 52s
2727:	learn: 4.3621582	test: 4.3188852	best: 4.3188852 (2727)	total: 18m 43s	remaining: 1m 52s
2728:	learn: 4.3621165	test: 4.3188471	best: 4.3188471 (2728)	total: 18m 43s	remaining: 1m 51s
2729:	learn: 4.3620538	test: 4.3187600	best: 4.3187600 (2729)	total: 18m 44s	remaining: 1m 51s
2730:	learn: 4.3620121	test: 4.3187268	best: 4.3187268 (2730)	total: 18m 44s	remaining: 1m 50s
2731:	learn: 4.3619961	test: 4.3187287	best: 4.3187268 (2730)	total: 18m 45s	remaining: 1m 50s
2732:	learn: 4.3619550	test: 4.3187056	best: 4.3187056 (2732)	total: 18m 45s	remaining: 1m 49s
2733:	learn: 4.3619270	test: 4.3187063	best: 4.3187056 (2732)	total: 18m 46s	remaining: 1m 49s
2734:	learn: 4.3618865	test: 4.3186990	best: 4.318

2811:	learn: 4.3590776	test: 4.3174833	best: 4.3174833 (2811)	total: 19m 19s	remaining: 1m 17s
2812:	learn: 4.3590466	test: 4.3174702	best: 4.3174702 (2812)	total: 19m 19s	remaining: 1m 17s
2813:	learn: 4.3590133	test: 4.3174681	best: 4.3174681 (2813)	total: 19m 20s	remaining: 1m 16s
2814:	learn: 4.3589670	test: 4.3174479	best: 4.3174479 (2814)	total: 19m 20s	remaining: 1m 16s
2815:	learn: 4.3589391	test: 4.3174548	best: 4.3174479 (2814)	total: 19m 21s	remaining: 1m 15s
2816:	learn: 4.3589093	test: 4.3174380	best: 4.3174380 (2816)	total: 19m 21s	remaining: 1m 15s
2817:	learn: 4.3588587	test: 4.3174531	best: 4.3174380 (2816)	total: 19m 22s	remaining: 1m 15s
2818:	learn: 4.3588357	test: 4.3174451	best: 4.3174380 (2816)	total: 19m 22s	remaining: 1m 14s
2819:	learn: 4.3587999	test: 4.3174182	best: 4.3174182 (2819)	total: 19m 22s	remaining: 1m 14s
2820:	learn: 4.3587533	test: 4.3173577	best: 4.3173577 (2820)	total: 19m 23s	remaining: 1m 13s
2821:	learn: 4.3587248	test: 4.3173397	best: 4.317

2899:	learn: 4.3559931	test: 4.3162821	best: 4.3162787 (2897)	total: 19m 57s	remaining: 41.3s
2900:	learn: 4.3559616	test: 4.3162652	best: 4.3162652 (2900)	total: 19m 58s	remaining: 40.9s
2901:	learn: 4.3559388	test: 4.3162704	best: 4.3162652 (2900)	total: 19m 58s	remaining: 40.5s
2902:	learn: 4.3559020	test: 4.3162738	best: 4.3162652 (2900)	total: 19m 58s	remaining: 40.1s
2903:	learn: 4.3558471	test: 4.3162433	best: 4.3162433 (2903)	total: 19m 59s	remaining: 39.6s
2904:	learn: 4.3557693	test: 4.3161893	best: 4.3161893 (2904)	total: 19m 59s	remaining: 39.2s
2905:	learn: 4.3557270	test: 4.3161730	best: 4.3161730 (2905)	total: 20m	remaining: 38.8s
2906:	learn: 4.3556940	test: 4.3161708	best: 4.3161708 (2906)	total: 20m	remaining: 38.4s
2907:	learn: 4.3556668	test: 4.3161643	best: 4.3161643 (2907)	total: 20m 1s	remaining: 38s
2908:	learn: 4.3556280	test: 4.3161585	best: 4.3161585 (2908)	total: 20m 1s	remaining: 37.6s
2909:	learn: 4.3555950	test: 4.3161479	best: 4.3161479 (2909)	total: 20m

2987:	learn: 4.3528565	test: 4.3149774	best: 4.3149765 (2986)	total: 20m 35s	remaining: 4.96s
2988:	learn: 4.3528007	test: 4.3149455	best: 4.3149455 (2988)	total: 20m 36s	remaining: 4.55s
2989:	learn: 4.3527773	test: 4.3149517	best: 4.3149455 (2988)	total: 20m 36s	remaining: 4.14s
2990:	learn: 4.3527414	test: 4.3149419	best: 4.3149419 (2990)	total: 20m 37s	remaining: 3.72s
2991:	learn: 4.3527112	test: 4.3149316	best: 4.3149316 (2991)	total: 20m 37s	remaining: 3.31s
2992:	learn: 4.3526804	test: 4.3149397	best: 4.3149316 (2991)	total: 20m 38s	remaining: 2.9s
2993:	learn: 4.3526461	test: 4.3149143	best: 4.3149143 (2993)	total: 20m 38s	remaining: 2.48s
2994:	learn: 4.3526003	test: 4.3148909	best: 4.3148909 (2994)	total: 20m 38s	remaining: 2.07s
2995:	learn: 4.3525604	test: 4.3148657	best: 4.3148657 (2995)	total: 20m 39s	remaining: 1.65s
2996:	learn: 4.3525104	test: 4.3148425	best: 4.3148425 (2996)	total: 20m 39s	remaining: 1.24s
2997:	learn: 4.3524873	test: 4.3148388	best: 4.3148388 (2997)

In [18]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))


4.314807987429045


,UID,JID,user_feature_1,user_feature_2,user_feature_3,user_feature_4,user_feature_5,user_feature_6,user_feature_7,user_feature_8,...,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100,Rating
InteractionID,,,,,,,,,,,,,,,,,,,,,
0,11228,39,0.041494,-0.019669,-0.046502,0.003684,-0.088271,-0.067943,-0.074454,0.021834,...,-0.255151,-0.170399,0.300889,-0.094352,0.370475,-0.054978,0.089917,-0.048792,-0.003394,0.885862
14,13992,39,0.074211,0.193894,0.018584,0.079793,0.202177,-0.046509,0.204888,-0.128389,...,-0.255151,-0.170399,0.300889,-0.094352,0.370475,-0.054978,0.089917,-0.048792,-0.003394,1.473757
230989,13100,39,0.035885,0.159322,-0.138248,0.033161,-0.016106,0.060540,0.043884,0.098515,...,-0.255151,-0.170399,0.300889,-0.094352,0.370475,-0.054978,0.089917,-0.048792,-0.003394,-1.437906
226103,11004,39,0.080895,0.480071,-0.035331,0.122641,0.495008,0.226571,0.610407,-0.071279,...,-0.255151,-0.170399,0.300889,-0.094352,0.370475,-0.054978,0.089917,-0.048792,-0.003394,-1.391423
42319,22191,39,-0.074489,-0.072138,-0.025149,-0.033814,-0.083488,0.043833,-0.197419,0.091711,...,-0.255151,-0.170399,0.300889,-0.094352,0.370475,-0.054978,0.089917,-0.048792,-0.003394,-1.960676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204228,8615,80,0.203848,-0.410903,-0.033596,-0.022550,-0.680779,-0.661681,-0.305348,-0.142337,...,-0.065841,-0.023535,0.224092,-0.169393,0.199443,0.147830,-0.135635,-0.156386,0.032155,-2.120842
131692,2409,80,-0.079079,0.275712,-0.033832,0.131563,0.034264,0.007515,0.161056,0.117608,...,-0.065841,-0.023535,0.224092,-0.169393,0.199443,0.147830,-0.135635,-0.156386,0.032155,-1.778240
134883,2966,80,0.151060,0.077917,-0.019202,0.143226,0.047397,-0.088291,0.009833,0.131578,...,-0.065841,-0.023535,0.224092,-0.169393,0.199443,0.147830,-0.135635,-0.156386,0.032155,-2.987959


In [30]:
test_pool = Pool(test_joke_df_nofactrating, cat_features=cat_features)

In [31]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost_svd0.9.csv')

,Rating
InteractionID,
0,0.885862
1,-4.439634
2,-1.687442
3,5.583215
4,4.710066


In [32]:
test_joke_df_nofactrating

,UID,JID,user_feature_1,user_feature_2,user_feature_3,user_feature_4,user_feature_5,user_feature_6,user_feature_7,user_feature_8,...,joke_feature_92,joke_feature_93,joke_feature_94,joke_feature_95,joke_feature_96,joke_feature_97,joke_feature_98,joke_feature_99,joke_feature_100,Rating
InteractionID,,,,,,,,,,,,,,,,,,,,,
0,11228,39,0.041494,-0.019669,-0.046502,0.003684,-0.088271,-0.067943,-0.074454,0.021834,...,-0.255151,-0.170399,0.300889,-0.094352,0.370475,-0.054978,0.089917,-0.048792,-0.003394,0.885862
1,21724,85,-0.105730,0.600935,0.081735,0.280295,0.461382,0.055102,0.320269,-0.082147,...,-0.419674,-0.068509,0.260702,-0.094782,0.160607,0.178213,-0.137030,-0.014625,-0.246719,-4.439634
2,16782,56,-0.046537,-0.124728,0.065352,0.036919,-0.228116,-0.152849,-0.376306,0.332513,...,-0.196697,0.186677,0.149783,-0.103317,0.576104,-0.114517,-0.063121,-0.169587,0.068116,-1.687442
3,12105,42,-0.125930,0.032269,0.031715,-0.065563,-0.155323,0.128054,-0.007486,-0.070116,...,-0.263738,-0.083895,0.261084,-0.148073,0.344654,-0.087479,-0.035381,-0.117311,-0.078073,5.583215
4,14427,2,0.007909,-0.235814,0.025480,0.026753,-0.117487,-0.210771,-0.316912,0.068459,...,-0.297996,-0.132594,0.337949,0.067280,0.285934,-0.153183,0.132184,0.000751,-0.057890,4.710066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
362086,3085,66,0.117056,0.521257,-0.056028,0.179373,0.480985,0.191488,0.613380,0.081868,...,-0.388854,-0.223858,0.288515,-0.120058,0.255402,-0.131640,0.165418,0.017114,-0.151924,2.315108
362087,13765,31,0.009805,-0.261898,0.203455,0.200472,0.025892,-0.362058,-0.187364,0.005845,...,-0.104571,0.031827,0.070764,-0.249606,-0.049928,0.356879,-0.233594,-0.127793,-0.015678,3.512332
362088,10341,29,-0.127947,-0.145092,-0.140193,-0.033378,-0.383222,-0.157345,-0.082459,-0.038048,...,-0.246149,-0.237770,0.365300,-0.070434,0.254199,0.160938,-0.345331,0.097235,0.231820,4.840404


In [25]:
parameters = {
    'iterations': 2000,
    'custom_metric': 'RMSE',
    'random_seed': 0,
    'train_dir':'RMSE',
    'objective':'RMSE',
    'loss_function':'RMSE',
    'eval_metric':'RMSE',
}

model = CatBoostRegressor(**parameters)
model.fit(main_pool, eval_set=valid_pool, plot=True)

C:\ProgramData\Anaconda3\envs\torchvision\lib\site-packages\catboost\core.py:6240: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.104598
0:	learn: 5.1153850	test: 5.1108192	best: 5.1108192 (0)	total: 306ms	remaining: 10m 10s
1:	learn: 5.0169725	test: 4.9991869	best: 4.9991869 (1)	total: 421ms	remaining: 7m
2:	learn: 4.9351641	test: 4.9052967	best: 4.9052967 (2)	total: 668ms	remaining: 7m 24s
3:	learn: 4.8665402	test: 4.8251139	best: 4.8251139 (3)	total: 792ms	remaining: 6m 34s
4:	learn: 4.8101678	test: 4.7604225	best: 4.7604225 (4)	total: 1.05s	remaining: 6m 58s
5:	learn: 4.7615142	test: 4.7018631	best: 4.7018631 (5)	total: 1.18s	remaining: 6m 31s
6:	learn: 4.7218404	test: 4.6536295	best: 4.6536295 (6)	total: 1.3s	remaining: 6m 11s
7:	learn: 4.6885635	test: 4.6130843	best: 4.6130843 (7)	total: 1.42s	remaining: 5m 54s
8:	learn: 4.6617920	test: 4.5800755	best: 4.5800755 (8)	total: 1.54s	remaining: 5m 41s
9:	learn: 4.6386253	test: 4.5504704	best: 4.5504704 (9)	total: 1.66s	remaining: 5m 31s
10:	learn: 4.6195351	test: 4.5258487	best: 4.5258487 (10)	total: 1.79s	remaining: 5m 23s
11:	learn: 4.60

94:	learn: 4.5093854	test: 4.3665878	best: 4.3665878 (94)	total: 11.9s	remaining: 3m 59s
95:	learn: 4.5093486	test: 4.3665316	best: 4.3665316 (95)	total: 12.1s	remaining: 3m 59s
96:	learn: 4.5093366	test: 4.3664927	best: 4.3664927 (96)	total: 12.2s	remaining: 3m 58s
97:	learn: 4.5093126	test: 4.3664823	best: 4.3664823 (97)	total: 12.3s	remaining: 3m 58s
98:	learn: 4.5092965	test: 4.3664501	best: 4.3664501 (98)	total: 12.4s	remaining: 3m 58s
99:	learn: 4.5092679	test: 4.3664226	best: 4.3664226 (99)	total: 12.6s	remaining: 3m 58s
100:	learn: 4.5092418	test: 4.3663775	best: 4.3663775 (100)	total: 12.7s	remaining: 3m 58s
101:	learn: 4.5092212	test: 4.3663342	best: 4.3663342 (101)	total: 12.8s	remaining: 3m 58s
102:	learn: 4.5091936	test: 4.3662820	best: 4.3662820 (102)	total: 12.9s	remaining: 3m 58s
103:	learn: 4.5090836	test: 4.3661440	best: 4.3661440 (103)	total: 13.1s	remaining: 3m 58s
104:	learn: 4.5090753	test: 4.3661231	best: 4.3661231 (104)	total: 13.2s	remaining: 3m 58s
105:	learn:

186:	learn: 4.5070333	test: 4.3641573	best: 4.3641573 (186)	total: 23.3s	remaining: 3m 46s
187:	learn: 4.5070082	test: 4.3641306	best: 4.3641306 (187)	total: 23.4s	remaining: 3m 45s
188:	learn: 4.5069896	test: 4.3641091	best: 4.3641091 (188)	total: 23.5s	remaining: 3m 45s
189:	learn: 4.5069788	test: 4.3641179	best: 4.3641091 (188)	total: 23.7s	remaining: 3m 45s
190:	learn: 4.5069686	test: 4.3641041	best: 4.3641041 (190)	total: 23.8s	remaining: 3m 45s
191:	learn: 4.5069344	test: 4.3640717	best: 4.3640717 (191)	total: 23.9s	remaining: 3m 45s
192:	learn: 4.5069267	test: 4.3640559	best: 4.3640559 (192)	total: 24s	remaining: 3m 44s
193:	learn: 4.5069008	test: 4.3640401	best: 4.3640401 (193)	total: 24.1s	remaining: 3m 44s
194:	learn: 4.5068582	test: 4.3640067	best: 4.3640067 (194)	total: 24.2s	remaining: 3m 44s
195:	learn: 4.5068508	test: 4.3640005	best: 4.3640005 (195)	total: 24.4s	remaining: 3m 44s
196:	learn: 4.5068324	test: 4.3639650	best: 4.3639650 (196)	total: 24.5s	remaining: 3m 44s
1

...

1631:	learn: 4.5009173	test: 4.3609919	best: 4.3609919 (1631)	total: 3m 18s	remaining: 44.7s
1632:	learn: 4.5009171	test: 4.3609922	best: 4.3609919 (1631)	total: 3m 18s	remaining: 44.6s
1633:	learn: 4.5009164	test: 4.3609918	best: 4.3609918 (1633)	total: 3m 18s	remaining: 44.4s
1634:	learn: 4.5009158	test: 4.3609929	best: 4.3609918 (1633)	total: 3m 18s	remaining: 44.3s
1635:	learn: 4.5009118	test: 4.3609919	best: 4.3609918 (1633)	total: 3m 18s	remaining: 44.2s
1636:	learn: 4.5009113	test: 4.3609921	best: 4.3609918 (1633)	total: 3m 18s	remaining: 44.1s
1637:	learn: 4.5009103	test: 4.3609922	best: 4.3609918 (1633)	total: 3m 18s	remaining: 43.9s
1638:	learn: 4.5009043	test: 4.3609913	best: 4.3609913 (1638)	total: 3m 18s	remaining: 43.8s
1639:	learn: 4.5009037	test: 4.3609919	best: 4.3609913 (1638)	total: 3m 19s	remaining: 43.7s
1640:	learn: 4.5009031	test: 4.3609878	best: 4.3609878 (1640)	total: 3m 19s	remaining: 43.6s
1641:	learn: 4.5009006	test: 4.3609897	best: 4.3609878 (1640)	total: 3

1721:	learn: 4.5007708	test: 4.3609699	best: 4.3609654 (1714)	total: 3m 29s	remaining: 33.8s
1722:	learn: 4.5007686	test: 4.3609693	best: 4.3609654 (1714)	total: 3m 29s	remaining: 33.7s
1723:	learn: 4.5007639	test: 4.3609698	best: 4.3609654 (1714)	total: 3m 29s	remaining: 33.6s
1724:	learn: 4.5007611	test: 4.3609706	best: 4.3609654 (1714)	total: 3m 29s	remaining: 33.5s
1725:	learn: 4.5007594	test: 4.3609701	best: 4.3609654 (1714)	total: 3m 30s	remaining: 33.3s
1726:	learn: 4.5007584	test: 4.3609642	best: 4.3609642 (1726)	total: 3m 30s	remaining: 33.2s
1727:	learn: 4.5007576	test: 4.3609657	best: 4.3609642 (1726)	total: 3m 30s	remaining: 33.1s
1728:	learn: 4.5007544	test: 4.3609637	best: 4.3609637 (1728)	total: 3m 30s	remaining: 33s
1729:	learn: 4.5007534	test: 4.3609616	best: 4.3609616 (1729)	total: 3m 30s	remaining: 32.9s
1730:	learn: 4.5007528	test: 4.3609592	best: 4.3609592 (1730)	total: 3m 30s	remaining: 32.7s
1731:	learn: 4.5007499	test: 4.3609599	best: 4.3609592 (1730)	total: 3m 

1810:	learn: 4.5006177	test: 4.3609105	best: 4.3609105 (1810)	total: 3m 40s	remaining: 23s
1811:	learn: 4.5006158	test: 4.3609136	best: 4.3609105 (1810)	total: 3m 40s	remaining: 22.9s
1812:	learn: 4.5006141	test: 4.3609128	best: 4.3609105 (1810)	total: 3m 40s	remaining: 22.8s
1813:	learn: 4.5006121	test: 4.3609114	best: 4.3609105 (1810)	total: 3m 41s	remaining: 22.7s
1814:	learn: 4.5006096	test: 4.3609155	best: 4.3609105 (1810)	total: 3m 41s	remaining: 22.5s
1815:	learn: 4.5006037	test: 4.3609149	best: 4.3609105 (1810)	total: 3m 41s	remaining: 22.4s
1816:	learn: 4.5006031	test: 4.3609183	best: 4.3609105 (1810)	total: 3m 41s	remaining: 22.3s
1817:	learn: 4.5006027	test: 4.3609173	best: 4.3609105 (1810)	total: 3m 41s	remaining: 22.2s
1818:	learn: 4.5005992	test: 4.3609177	best: 4.3609105 (1810)	total: 3m 41s	remaining: 22.1s
1819:	learn: 4.5005968	test: 4.3609170	best: 4.3609105 (1810)	total: 3m 41s	remaining: 21.9s
1820:	learn: 4.5005912	test: 4.3609149	best: 4.3609105 (1810)	total: 3m 

1900:	learn: 4.5004751	test: 4.3609248	best: 4.3609103 (1836)	total: 3m 51s	remaining: 12.1s
1901:	learn: 4.5004745	test: 4.3609277	best: 4.3609103 (1836)	total: 3m 52s	remaining: 12s
1902:	learn: 4.5004744	test: 4.3609273	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.8s
1903:	learn: 4.5004740	test: 4.3609258	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.7s
1904:	learn: 4.5004724	test: 4.3609241	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.6s
1905:	learn: 4.5004694	test: 4.3609239	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.5s
1906:	learn: 4.5004662	test: 4.3609239	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.3s
1907:	learn: 4.5004645	test: 4.3609259	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.2s
1908:	learn: 4.5004614	test: 4.3609193	best: 4.3609103 (1836)	total: 3m 52s	remaining: 11.1s
1909:	learn: 4.5004612	test: 4.3609178	best: 4.3609103 (1836)	total: 3m 53s	remaining: 11s
1910:	learn: 4.5004602	test: 4.3609193	best: 4.3609103 (1836)	total: 3m 53

1989:	learn: 4.5003237	test: 4.3608968	best: 4.3608933 (1980)	total: 4m 2s	remaining: 1.22s
1990:	learn: 4.5003232	test: 4.3608969	best: 4.3608933 (1980)	total: 4m 2s	remaining: 1.1s
1991:	learn: 4.5003229	test: 4.3608978	best: 4.3608933 (1980)	total: 4m 3s	remaining: 976ms
1992:	learn: 4.5003228	test: 4.3608984	best: 4.3608933 (1980)	total: 4m 3s	remaining: 854ms
1993:	learn: 4.5003220	test: 4.3608975	best: 4.3608933 (1980)	total: 4m 3s	remaining: 732ms
1994:	learn: 4.5003212	test: 4.3608934	best: 4.3608933 (1980)	total: 4m 3s	remaining: 610ms
1995:	learn: 4.5003179	test: 4.3608869	best: 4.3608869 (1995)	total: 4m 3s	remaining: 488ms
1996:	learn: 4.5003145	test: 4.3608862	best: 4.3608862 (1996)	total: 4m 3s	remaining: 366ms
1997:	learn: 4.5003127	test: 4.3608901	best: 4.3608862 (1996)	total: 4m 3s	remaining: 244ms
1998:	learn: 4.5003106	test: 4.3608879	best: 4.3608862 (1996)	total: 4m 3s	remaining: 122ms
1999:	learn: 4.5003102	test: 4.3608895	best: 4.3608862 (1996)	total: 4m 4s	remain

In [26]:
predict = model.predict(valid_pool)
print(mean_squared_error(valid_df['Rating'].values, predict, squared=False))


4.360886211442052


In [27]:
predict = model.predict(test_pool)

test_joke_df_nofactrating['Rating'] = predict

display(test_joke_df_nofactrating['Rating'].to_frame().head(5))
test_joke_df_nofactrating['Rating'].to_frame().to_csv('catboost.csv')

,Rating
InteractionID,
0,1.088325
1,-2.773870
2,-1.315627
3,5.417442
4,4.914472
